In [1]:
import torch
import torch.nn as nn

class MaskedLinear(nn.Linear):
    def __init__(self, in_features, out_features, bias=True):
        super(MaskedLinear, self).__init__(in_features, out_features, bias)
        self.register_buffer('weight_mask', torch.ones(out_features, in_features))

    def forward(self, input):
        masked_weight = self.weight * self.weight_mask
        return nn.functional.linear(input, masked_weight, self.bias)

# Create a simple model with a masked linear layer
model = nn.Sequential(
    MaskedLinear(3, 4),
    nn.ReLU(),
    nn.Linear(4, 2)
)

# Set a specific weight mask for the masked linear layer
new_mask = torch.tensor([
    [1, 0, 1],
    [1, 1, 0],
    [0, 1, 1],
    [1, 1, 1]
])
model[0].weight_mask.copy_(new_mask)

tensor([[1., 0., 1.],
        [1., 1., 0.],
        [0., 1., 1.],
        [1., 1., 1.]])

In [7]:
model[0].weight_mask

tensor([[1., 0., 1.],
        [1., 1., 0.],
        [0., 1., 1.],
        [1., 1., 1.]])

In [16]:
import torch
import torch.nn as nn

class SimpleFeedforward(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleFeedforward, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        return x

input_size = 3
hidden_size = 4
output_size = 2

model = SimpleFeedforward(input_size, hidden_size, output_size)
model_state_dict = model.state_dict()
print(model_state_dict)

OrderedDict([('hidden.weight', tensor([[-0.0608,  0.0539,  0.5331],
        [-0.4810,  0.1233,  0.0555],
        [-0.4860,  0.0546,  0.0107],
        [-0.2816,  0.4855, -0.0054]])), ('hidden.bias', tensor([ 0.1430,  0.2492, -0.4270,  0.0581])), ('output.weight', tensor([[ 0.3120, -0.1003,  0.1688,  0.1702],
        [-0.3985, -0.1890, -0.0265,  0.1303]])), ('output.bias', tensor([0.4874, 0.4113]))])


In [18]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0608,  0.0539,  0.5331],
         [-0.4810,  0.1233,  0.0555],
         [-0.4860,  0.0546,  0.0107],
         [-0.2816,  0.4855, -0.0054]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1430,  0.2492, -0.4270,  0.0581], requires_grad=True),
 Parameter containing:
 tensor([[ 0.3120, -0.1003,  0.1688,  0.1702],
         [-0.3985, -0.1890, -0.0265,  0.1303]], requires_grad=True),
 Parameter containing:
 tensor([0.4874, 0.4113], requires_grad=True)]

In [19]:
import torch.optim as optim
optim.SGD(model.parameters(), lr=1e-10).zero_grad()

In [20]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0608,  0.0539,  0.5331],
         [-0.4810,  0.1233,  0.0555],
         [-0.4860,  0.0546,  0.0107],
         [-0.2816,  0.4855, -0.0054]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1430,  0.2492, -0.4270,  0.0581], requires_grad=True),
 Parameter containing:
 tensor([[ 0.3120, -0.1003,  0.1688,  0.1702],
         [-0.3985, -0.1890, -0.0265,  0.1303]], requires_grad=True),
 Parameter containing:
 tensor([0.4874, 0.4113], requires_grad=True)]

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader

class RandomTensorDataset(Dataset):
    def __init__(self, num_samples=100, input_size=10):
        self.num_samples = num_samples
        self.input_size = input_size
        self.data = torch.randn(num_samples, input_size)
        self.targets = torch.randint(0, 2, (num_samples,))

    def __len__(self):
        return self.num_samples

    def __getitem__(self, index):
        return self.data[index], self.targets[index]

# Create a custom dataset with 100 samples, each with 10 features
random_dataset = RandomTensorDataset(num_samples=100, input_size=10)

# Create a DataLoader with a batch size of 20
dummy_loader = DataLoader(random_dataset, batch_size=20, shuffle=True)

# Loop through the DataLoader
for batch_idx, (in_tensor, target) in enumerate(dummy_loader):
    print(f"Batch {batch_idx}:")
    print(f"Input tensor (in_tensor) shape: {in_tensor.shape}")
    print(f"Target tensor shape: {target.shape}")
    print()


Batch 0:
Input tensor (in_tensor) shape: torch.Size([20, 10])
Target tensor shape: torch.Size([20])

Batch 1:
Input tensor (in_tensor) shape: torch.Size([20, 10])
Target tensor shape: torch.Size([20])

Batch 2:
Input tensor (in_tensor) shape: torch.Size([20, 10])
Target tensor shape: torch.Size([20])

Batch 3:
Input tensor (in_tensor) shape: torch.Size([20, 10])
Target tensor shape: torch.Size([20])

Batch 4:
Input tensor (in_tensor) shape: torch.Size([20, 10])
Target tensor shape: torch.Size([20])

